# SCKAN Example Queries in SPARQL
#### This notebook provides a set of example queries to explore the SCKAN knowledge base using SPARQL. It is recommended that you have the basic understanding of how SCKAN models its connectivity based on the documentation [linked here](https://github.com/smtifahim/sckan-query-examples/blob/main/README.md).
#### To use this notebook you will need to install [jupyter notebook](https://jupyter.org/install) with [SPARQL kernel](https://github.com/paulovn/sparql-kernel). Run the following commands in your terminal.
* Install Jupyter Notebook: `pip install notebook`
    * To run Jupyter Notebook: `jupyter notebook`
* Install the python package: `pip install sparqlkernel`
    * Install the kernel into Jupyter: `jupyter sparqlkernel install`
After the installations, download this sckan-sparql-query-examples.ipynb notebook file and open it in your Jupyter Notebook. 

The comments within the SPARQL queries should provide information on how to customize the queries. You can copy and paste (edit as needed) any of the queries as a query string in your python code to retreive data.

<b>Note:</b> While the notebook approach is useful for documentation purposes, it could be a pain using Jupyter Notebook to play around with SPARQL queries as it is not optimized enough for editing queries. If you prefer a more flexible SPARQL editor, try <b>YASGUI</b> (https://yasgui.triply.cc/) (Yet Another SPARQL GUI). No registration, installation, or setup is needed. Simply specify our blazegraph endpoint and try editing different queries from this notebook. You can download the query results in CSV or in raw JSON from YASGUI as well.
* <b>Blazegraph Endpoint</b>: https://blazegraph.scicrunch.io/blazegraph/sparql

## Set up the Blazegraph SPARQL Endpoint for SCKAN 
First, set up the SPARQL endpoint using following magics. To see the list of other available magics run `%lsmagics` in a code section.

In [6]:
%endpoint https://blazegraph.scicrunch.io/blazegraph/sparql
%format any
%display table
%show 16 
# The "magics" don't have to be in a separate cell from the actual query. 

Endpoint set to: https://blazegraph.scicrunch.io/blazegraph/sparql
Request format: ANY
Display: table
Result maximum size: 16

## Set up the Stardog SPARQL Endpoint for SCKAN
Alternatively, to set up the Stardog SPARQL Endpoint, you can use the following magic and hit 'Run'.

In [2]:
%endpoint https://sd-c1e74c63.stardog.cloud:5820/NPO/query
%auth basic SPARC RCvp9tKzTdxg42py
%format any
%display table
%show 16

Endpoint set to: https://sd-c1e74c63.stardog.cloud:5820/NPO/query
HTTP authentication: method=basic, user=SPARC, passwd set
Request format: ANY
Display: table
Result maximum size: 16

In [2]:
%lsmagics

Available magics:
%auth  %display  %endpoint  %format  %graph  %header  %http_header  %lang  %load  %log  %lsmagics  %method  %outfile  %prefix  %qparam  %show

%auth (basic|digest|none) <username> <passwd> : send HTTP authentication (use env:<var> to get values from environment variables)
%display raw | table [withtypes] | diagram [svg|png] [withliterals] : set display format
%endpoint <url> : set SPARQL endpoint. **REQUIRED**
%format JSON | N3 | XML | default | any | none : set requested result format
%graph <uri> : set default graph for the queries
%header <string> | OFF : add a persistent SPARQL header line before all queries, or delete all defined headers
%http_header <name> [<value>] : add (or delete) an arbitrary HTTP header to all queries
%lang <lang> [...] | default | all : language(s) preferred for labels
%load <filename> : load a file with magic lines and process them
%log critical | error | warning | info | debug : set logging level
%lsmagics  : list all magics
%method get | post : set HTTP method
%outfile <filename> | off : save raw output to a file (use "%d" in name to add cell number, "off" to cancel saving)
%prefix <name> [<uri>] : set (or delete) a persistent URI prefix for all queries
%qparam <name> [<value>] : add (or delete) a persistent custom parameter to all queries
%show <n> | all : maximum number of shown results

## Example 0: All Neuron Populations
An example query to list all the neuron populations available in NPO along with their labels and preferred labels (if any).

In [6]:
# %format JSON
# %outfile example-0.json
# This query returns the list of neuron populations in SCKAN with their labels and preferred labels (when available). 
# The query considers each population to have at least one origin and destination locations.

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT 
# ?Neuron_ID 
?Neuron_Label ?Neuron_Preferred_Label
{
    ## For partial matching of neuron label or ID. # Replace 'liver' within quotes.
    FILTER (CONTAINS (LCase(?Neuron_Label), 'prostate'))
        
    # We are only interested in connection forming neuron types i.e, neurons that form some connection 
    # between their origin and destination regions
    ?Neuron_ID ilxtr:hasSomaLocation ?A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
    
    ?Neuron_ID rdfs:label ?Neuron_Label;
    OPTIONAL {?Neuron_ID skos:prefLabel ?Neuron_Preferred_Label.}
}
ORDER BY ?Neuron_ID ?Neuron_Label
LIMIT 999

Neuron_Label,Neuron_Preferred_Label
neuron type prostate 11,dorsal gray commissure and the intermediolateral column of T13-L2 spinal cord to major pelvic ganglion via inferior mesenteric nerve plexus via hypogastric nerve
neuron type prostate 12,major pelvic ganglion to prostate gland smooth muscle via postganglionic sympathetic fiber
neuron type prostate 13,intermediolateral cell column of L6-S1 spinal cord to major pelvic ganglion via pelvic splanchnic nerve
neuron type prostate 14,major pelvic ganglion to prostate epithelium via postganglionic parasympathetic fiber
neuron type prostate 15,intermediolateral column of thoracolumbar T11-L2 spinal cord to pelvic ganglion via superior hypogastric nerve plexus via hypogastric nerve
neuron type prostate 16,pelvic ganglion to prostate gland smooth muscle via postganglionic sympathetic fiber
neuron type prostate 17,intermediolateral cell column of sacral S2-S4 spinal cord to pelvic ganglion via pelvic splanchnic nerve
neuron type prostate 18,pelvic ganglion to prostate epithelium via postganglionic parasympathetic fiber
neuron type prostate 24,L2-L5 spinal cord to inferior mesenteric ganglion via lumbar splanchic nerve
neuron type prostate 25,inferior mesenteric ganglion to prostate gland smooth muscle via hypogastric nerve via postganglionic sympathetic fiber


## Neurons with Metadata
This is a query to list neuron populations with their available metadata in SCKAN such as species, sex, phenotype, and reference whenever available.  

In [7]:
# Query to retreive metadata of the neuron populations.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT 
# ?Neuron_IRI
?Neuron_Label ?Neuron_Pref_Label
(GROUP_CONCAT (DISTINCT ?Species_X; SEPARATOR = ", ") AS ?Species)
?Sex 
(GROUP_CONCAT (DISTINCT ?Phenotype; SEPARATOR = ", ") AS ?Phenotypes)
(GROUP_CONCAT (DISTINCT ?Reference; SEPARATOR= ", ") AS ?References)
WHERE
{
    ## Example: Filter the populations from prostate connections
    FILTER (CONTAINS (STR(?Neuron_IRI), "prostate"))

    ## Only considering connection forming neuron populations
    ?Neuron_IRI ilxtr:hasSomaLocation ?A_IRI; 
                (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B_IRI.
    
    OPTIONAL { ?Neuron_IRI rdfs:label ?Neuron_Label }
    OPTIONAL { ?Neuron_IRI skos:prefLabel ?Neuron_Pref_Label }         
    OPTIONAL { ?Neuron_IRI ilxtr:isObservedInSpecies/rdfs:label ?Species_X }
    OPTIONAL { ?Neuron_IRI ilxtr:hasPhenotypicSex/rdfs:label ?Sex }
    OPTIONAL { ?Neuron_IRI (ilxtr:hasNeuronalPhenotype | 
                            ilxtr:hasFunctionalCircuitRole |
                            ilxtr:hasCircuitRole |
                            ilxtr:hasProjection  
                            )/rdfs:label ?Phenotype }
    OPTIONAL { ?Neuron_IRI ilxtr:hasForwardConnection/rdfs:label ?Forward_Connection }
    OPTIONAL { ?Neuron_IRI ilxtr:alertNote ?Alert }
    OPTIONAL { ?Neuron_IRI ilxtr:literatureCitation ?Reference }
}
GROUP BY ?Neuron_IRI ?Neuron_Label ?Neuron_Pref_Label ?Species ?Sex ?Alert #?Reference ?Phenotype ?Forward_Connection
ORDER BY ?Neuron_IRI ?Neuron_Label
LIMIT 500

Neuron_Label,Neuron_Pref_Label,Species,Sex,Phenotypes,References
neuron type prostate 11,dorsal gray commissure and the intermediolateral column of T13-L2 spinal cord to major pelvic ganglion via inferior mesenteric nerve plexus via hypogastric nerve,Rattus norvegicus,male,Sympathetic Pre-Ganglionic phenotype,https://doi.org/10.1016/s0165-1838(98)00079-4
neuron type prostate 12,major pelvic ganglion to prostate gland smooth muscle via postganglionic sympathetic fiber,Rattus norvegicus,male,Sympathetic Post-Ganglionic phenotype,"https://doi.org/10.1007/BF00417871, https://doi.org/10.1016/s0166-4328(99)00084-4, https://doi.org/10.1186/1471-2202-3-8"
neuron type prostate 13,intermediolateral cell column of L6-S1 spinal cord to major pelvic ganglion via pelvic splanchnic nerve,Rattus norvegicus,male,Parasympathetic Pre-Ganglionic phenotype,"https://doi.org/10.1016/s0165-1838(98)00079-4, https://doi.org/10.1007/BF00417871, https://doi.org/10.1186/1471-2202-3-8"
neuron type prostate 14,major pelvic ganglion to prostate epithelium via postganglionic parasympathetic fiber,Rattus norvegicus,male,Parasympathetic Post-Ganglionic phenotype,"https://doi.org/10.1007/BF00417871, https://doi.org/10.1016/s0163-7258(02)00174-2, https://doi.org/10.1046/j.1365-2680.2000.00195.x"
neuron type prostate 15,intermediolateral column of thoracolumbar T11-L2 spinal cord to pelvic ganglion via superior hypogastric nerve plexus via hypogastric nerve,Homo sapiens,male,Sympathetic Pre-Ganglionic phenotype,"https://doi.org/10.1002/ca.10187, https://doi.org/10.1016/b978-0-7020-3131-1.00030-4, https://doi.org/10.1590/s0041-87812002000600008"
neuron type prostate 16,pelvic ganglion to prostate gland smooth muscle via postganglionic sympathetic fiber,Homo sapiens,male,Sympathetic Post-Ganglionic phenotype,"https://doi.org/10.1016/s0163-7258(02)00174-2, https://doi.org/10.1046/j.1365-2680.2000.00195.x, https://doi.org/10.4161/org.24843"
neuron type prostate 17,intermediolateral cell column of sacral S2-S4 spinal cord to pelvic ganglion via pelvic splanchnic nerve,Homo sapiens,male,Parasympathetic Pre-Ganglionic phenotype,"https://doi.org/10.1002/ca.10187, https://doi.org/10.1016/b978-0-7020-3131-1.00030-4, https://doi.org/10.1590/s0041-87812002000600008"
neuron type prostate 18,pelvic ganglion to prostate epithelium via postganglionic parasympathetic fiber,Homo sapiens,male,Parasympathetic Post-Ganglionic phenotype,"https://doi.org/10.1046/j.1365-2680.2000.00195.x, https://doi.org/10.4161/org.24843"
neuron type prostate 24,L2-L5 spinal cord to inferior mesenteric ganglion via lumbar splanchic nerve,Canis familiaris,male,Sympathetic Pre-Ganglionic phenotype,https://doi.org/10.1002/1097-0045(20000801)44:3%3C225::AID-PROS7%3E3.0.CO;2-G
neuron type prostate 25,inferior mesenteric ganglion to prostate gland smooth muscle via hypogastric nerve via postganglionic sympathetic fiber,Canis familiaris,male,Sympathetic Post-Ganglionic phenotype,"https://doi.org/10.1046/j.1365-2680.2000.00195.x, https://doi.org/10.1016/s0022-5347(17)42128-8"


## List of Phenotypic Properties used in SCKAN
Please refer to the [readme](https://github.com/smtifahim/sckan-query-examples/blob/main/README.md) document to know more about these relational properties.

In [8]:
## This query returns the list of phenotypic properties and the count of their use 
## in SCKAN neuron populations.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>

SELECT DISTINCT ?phenotype (count (distinct ?neuron) as ?count)
WHERE 
{
    ?phenotype rdf:type owl:AnnotationProperty.
    ?phenotype rdfs:subPropertyOf+ ilxtr:hasSimpleSckanProperty.
    # ?phenotype rdfs:label ?phenotype_label.

    ?neuron ?phenotype ?o.

    ?neuron rdfs:subClassOf* ?sckan_neuron .
    values ?sckan_neuron { ilxtr:NeuronSparcNlp ilxtr:NeuronApinatSimple }
    
    # Excluding the following two classification properties
    Filter (?phenotype != ilxtr:hasNonLocationalPhenotype && ?phenotype != ilxtr:hasConnectedLocation) 
}
GROUP BY ?phenotype # ?phenotype_label
ORDER BY desc (?count)

phenotype,count
http://uri.interlex.org/tgbugs/uris/readable/hasSomaLocation,463
http://uri.interlex.org/tgbugs/uris/readable/hasAxonTerminalLocation,461
http://uri.interlex.org/tgbugs/uris/readable/hasAxonLocation,429
http://uri.interlex.org/tgbugs/uris/readable/isObservedInSpecies,375
http://uri.interlex.org/tgbugs/uris/readable/hasCircuitRole,224
http://uri.interlex.org/tgbugs/uris/readable/hasNeuronalPhenotype,209
http://uri.interlex.org/tgbugs/uris/readable/hasForwardConnection,108
http://uri.interlex.org/tgbugs/uris/readable/hasPhenotypicSex,96
http://uri.interlex.org/tgbugs/uris/readable/hasAxonSensoryLocation,74
http://uri.interlex.org/tgbugs/uris/readable/hasAxonToSensoryTerminal,69


## Example 1: Neurons at A projects to B
The most basic query we want to have for SCKAN is to list the neuron populations along with their origins and destonations.  

In [18]:
# Query: return the neuron populations where Neuron at A projects to B. 
# In this case, A refers to the soma location of the neuron population  and B refers to the axon terminal 
# or the axon sensory terminal location of the neuron population.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ILX: <http://uri.interlex.org/base/ilx_>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT 
# ?Neuron_ID 
?Neuron_Label 
?A 
?Region_A
?B
?Region_B
{
  
  ## ***************************** INSTRUCTIONS **************************************** 
  ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
  ## REPLACE the region name on the right side of the equality operator (=) 
  ## by your desired region name within quotes (all in lower case).
  ## ***********************************************************************************

  # Example: What populations originate from 'inferior vagus X ganglion' # and project to 'nucleus of solitary tract'
  FILTER REGEX ( LCase(?Region_A), 'inferior vagus x ganglion' ) # For partial matching of Region_A labels  
  # FILTER REGEX ( LCase(?Region_B), 'nucleus of solitary tract' ) # For partial matching of Region_B labels  
  
  # FILTER REGEX (?Neuron_Label, 'aacar') # For partial matching of neuron labels

   ?Neuron_ID rdfs:label ?Neuron_Label;
              ilxtr:hasSomaLocation ?A;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
 
   ?A rdfs:label ?Region_A. ?B rdfs:label ?Region_B.
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B
LIMIT 1000 

Neuron_Label,A,Region_A,B,Region_B
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0000947,aorta
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002348,epicardium
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002084,heart left ventricle
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002080,heart right ventricle
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002079,left cardiac atrium
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002012,pulmonary artery
neuron type aacar 11,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0002078,right cardiac atrium
neuron type bolew unbranched 10,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://uri.interlex.org/base/ilx_0738318,Epiglottic mechanoreceptors
neuron type bolew unbranched 10,http://purl.obolibrary.org/obo/UBERON_0005363,inferior vagus X ganglion,http://purl.obolibrary.org/obo/UBERON_0009050,nucleus of solitary tract


## Example 2: Neuron Population at A projects to B (with synonyms)
This is the same query as before except that we can filter through any synonyms of the regions instead of just exact labels.

In [17]:
# Query: return the neuron populations where Neuron at A projects to B with synonyms for A and B. 
# In this case, A refers to the soma location of the neuron population  and B refers to the axon terminal 
# or the axon sensory terminal location of the neuron population. 

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

SELECT DISTINCT #?Neuron_ID 
?Neuron_Label
# ?A 
?Region_A
# ?B
?Region_B
{
  
  ## ***************************** INSTRUCTIONS **************************************** 
  ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
  ## REPLACE the region name on the right side of the equality operator (=) 
  ## by your desired region name within quotes (all in lower case).
  ## ***********************************************************************************

  # For partial matching of Region_A labels
  # Example: What populations originate from 'nodose ganglion' # and project to 'epicardium' region
  FILTER REGEX ( LCase(?Region_A_Synonym), 'nodose ganglion' )   # using a synonym for inferior vagus X ganglion
 # FILTER REGEX ( LCase(?Region_B_Synonym), 'nucleus of solitary tract' ) # For partial matching of Region_B labels  
  
  # FILTER REGEX (?Neuron_Label, 'aacar') # For partial matching of neuron labels

   ?Neuron_ID rdfs:label ?Neuron_Label;
              ilxtr:hasSomaLocation ?A;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
 
   ?A (rdfs:label | oboInOwl:hasExactSynonym) ?Region_A_Synonym. 
   ?B (rdfs:label | oboInOwl:hasExactSynonym) ?Region_B_Synonym.

   ?A rdfs:label ?Region_A.
   ?B rdfs:label ?Region_B.
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B
LIMIT 9999

Neuron_Label,Region_A,Region_B
neuron type aacar 11,inferior vagus X ganglion,aorta
neuron type aacar 11,inferior vagus X ganglion,epicardium
neuron type aacar 11,inferior vagus X ganglion,heart left ventricle
neuron type aacar 11,inferior vagus X ganglion,heart right ventricle
neuron type aacar 11,inferior vagus X ganglion,left cardiac atrium
neuron type aacar 11,inferior vagus X ganglion,nucleus of solitary tract
neuron type aacar 11,inferior vagus X ganglion,pulmonary artery
neuron type aacar 11,inferior vagus X ganglion,right cardiac atrium
neuron type bolew unbranched 10,inferior vagus X ganglion,Epiglottic mechanoreceptors
neuron type bolew unbranched 10,inferior vagus X ganglion,nucleus of solitary tract


## Example 3: Neurons at A projects to B via C
One of the most important queries we frequently need is to list the neuron populations along with their origin (A), destination (B), and via locations (C). 

In [12]:
# QUERY: List the Neuron Populations where Neurons at Region A Projects to B via C.
# This query returns any neuron population that has its soma located in Region A,
#                                                       axon terminal or axon sensory terminal located in Region B, and 
#                                                       axon located in region C.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>

SELECT DISTINCT 
#?Neuron_ID 
?Neuron_Label ?Region_A ?Region_B ?Region_C
{
   ## ***************************** INSTRUCTIONS **************************************** 
   ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
   ## REPLACE the region name on the right side of the equality operator (=) 
   ## by your desired region name within quotes (all in lower case).
   ## ***********************************************************************************  

   ## For exact matching of the regions names
     FILTER ( LCase(?Region_A) = 'nucleus ambiguus' )
   # FILTER ( LCase(?Region_B) = 'bronchiole parasympathetic ganglia' )
   # FILTER ( LCase(?Region_C) = 'vagus nerve' )
   
   ## For partial matching of region names (label or synonyms)
   # FILTER REGEX ( LCase(?Region_A), 'ambiguus' )
   # FILTER REGEX ( LCase(?Region_B), 'bronchiole parasympathetic' ) 
   # FILTER REGEX ( LCase(?Region_C), 'vagus' )
   
   # FILTER REGEX ( LCase(?Neuron_Label), 'bolew unbranched 25' ) # For partial matching of neuron label
                                                      # Replace 'bolew unbranched 25' within quotes.  

   ?Neuron_ID rdfs:label ?Neuron_Label;
               ilxtr:hasSomaLocation ?A;
               ilxtr:hasAxonLocation ?C;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B. 
    
          ?A rdfs:label ?Region_A.
          ?B rdfs:label ?Region_B.
          ?C rdfs:label ?Region_C. 
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B ?Region_C
LIMIT 5000

Neuron_Label,Region_A,Region_B,Region_C
neuron type aacar 4,nucleus ambiguus,Atrial intrinsic cardiac ganglion,medulla oblongata
neuron type aacar 4,nucleus ambiguus,Atrial intrinsic cardiac ganglion,vago-sympathetic trunk
neuron type aacar 4,nucleus ambiguus,Atrial intrinsic cardiac ganglion,vagus nerve
neuron type aacar 4,nucleus ambiguus,ventricular intrinsic cardiac ganglion,medulla oblongata
neuron type aacar 4,nucleus ambiguus,ventricular intrinsic cardiac ganglion,vago-sympathetic trunk
neuron type aacar 4,nucleus ambiguus,ventricular intrinsic cardiac ganglion,vagus nerve
neuron type bromo 2,nucleus ambiguus,bronchiole parasympathetic ganglia,vagus nerve
neuron type bromo 2,nucleus ambiguus,bronchus parasympathetic ganglia,vagus nerve
neuron type bromo 2,nucleus ambiguus,terminal bronchiole parasympathetic ganglia,vagus nerve
neuron type bromo 2,nucleus ambiguus,trachea parasympathetic ganglia,vagus nerve


### Example 4: Neurons at A projects to B via C (with synonyms)
This is the same query as the previous except that we can now filter through any synonyms of the regions instead of just exact labels. 

In [ ]:
# QUERY: List the Neuron Populations where Neuron at Region A Projects to B via C.
# This query returns any neuron population that has its soma located in Region A,
#                                                       axon terminal or axon sensory terminal located in Region B, and 
#                                                       axon located in region C.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#> 

SELECT DISTINCT ?Neuron_ID ?Neuron_Label ?A ?Region_A ?B ?Region_B ?C ?Region_C
{
   ## ***************************** INSTRUCTIONS **************************************** 
   ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
   ## REPLACE the region name on the right side of the equality operator (=) 
   ## by your desired region name within quotes (all in lower case).
   ## ***********************************************************************************  

   ## For exact matching of the regions names
   # FILTER ( LCase(?Region_A) = 'nucleus ambiguus' )
   # FILTER ( LCase(?Region_B) = 'bronchiole parasympathetic ganglia' )
   # FILTER ( LCase(?Region_C) = 'vagus nerve' )
   
   ## For partial matching of region names (label or synonyms)
   # FILTER REGEX ( LCase(?Region_A), 'ambiguus' )
   # FILTER REGEX ( LCase(?Region_B), 'bronchiole parasympathetic' ) 
   # FILTER REGEX ( LCase(?Region_C), 'vagus' )
   
   ## For partial matching of neuron label. Replace 'splen 4' within quotes.
   # FILTER REGEX (?Neuron_Label, 'splen 4') 
   
   ?Neuron_ID rdfs:label ?Neuron_Label;
               ilxtr:hasSomaLocation ?A;
               ilxtr:hasAxonLocation ?C;
              (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B. 
    
    ?A (rdfs:label | oboInOwl:hasExactSynonym) ?Region_A.
    ?B (rdfs:label | oboInOwl:hasExactSynonym) ?Region_B.
    ?C (rdfs:label | oboInOwl:hasExactSynonym) ?Region_C.
}
ORDER BY ?Neuron_ID ?Region_A ?Region_B ?Region_C
LIMIT 500

## Example 5: Neurons at A projects to B via Nerve C
This is another important query for SCKAN where we want to know the neuronal connections based on their via locations in nerves rather than just any axon locations.

In [13]:
# QUERY: List the Neuron Populations where Neuron at Region A Projects to B via Nerve C.
# This query returns any neuron population that has its soma located in Region A,
#                                                       axon terminal located in Region B, and 
#                                                       axon located in nerve C.
# Note that region C could be of type nerve, nerve plexus, or nerve fiber from UBERON.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>

# Neuron populations where A projects to B via some Nerve C

SELECT DISTINCT 
#?Neuron_ID 
?Neuron_Label ?Region_A ?Region_B ?Region_C 
{
   ## ***************************** INSTRUCTIONS **************************************** 
   ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
   ## REPLACE the region name on the right side of the equality operator (=) 
   ## by your desired region name within quotes (all in lower case).
   ## ***********************************************************************************
 
   ## For exact matching of region names 
   # FILTER ( LCase(?Region_A) = 'inferior vagus x ganglion' )
   # FILTER ( LCase(?Region_B) = 'nucleus of solitary tract' )
   FILTER ( LCase(?Region_C) = 'vagus nerve' ) # list the populations/connections via vagus nerve.
   
   ## For partial matching of region names 
   # FILTER REGEX (LCase(?Region_A), 'vagus x' )
   # FILTER REGEX (LCase(?Region_B), 'aortic' )
   #  FILTER REGEX (LCase(?Region_C), 'vagus' )
   
   ## For partial matching of neuron label pr ID. # Replace 'splen 4' within quotes.
   # FILTER REGEX (?Neuron_Label, 'splen 4')
   
    ?Neuron_ID rdfs:label ?Neuron_Label;
       ilxtr:hasSomaLocation ?A;
       ilxtr:hasAxonLocation ?C;
      (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.
    
    ?A rdfs:label ?Region_A.
    ?B rdfs:label ?Region_B.
    ?C rdfs:label ?Region_C.
    
    ## Check if region C is a subclass+ of a 'nerve' or 'nerve plexus'.
    ?C (rdfs:subClassOf+/rdfs:label) | rdfs:label ?nerve_label.  
    ## We have FMA 'Nerve' and UBERON 'nerve'. Also including UBERON 'nerve plexus'.
    FILTER (?nerve_label in ('nerve', 'nerve plexus', 'nerve fiber'))
                   # || CONTAINS(LCASE(?nerve_label), 'nerve'))
	
}
ORDER BY ?Neuron_ID ?Region_B ?Region_C
LIMIT 5000

Neuron_Label,Region_A,Region_B,Region_C
neuron type aacar 4,nucleus ambiguus,Atrial intrinsic cardiac ganglion,vagus nerve
neuron type aacar 4,dorsal motor nucleus of vagus nerve,Atrial intrinsic cardiac ganglion,vagus nerve
neuron type aacar 4,dorsal motor nucleus of vagus nerve,ventricular intrinsic cardiac ganglion,vagus nerve
neuron type aacar 4,nucleus ambiguus,ventricular intrinsic cardiac ganglion,vagus nerve
neuron type bolew unbranched 10,inferior vagus X ganglion,Epiglottic mechanoreceptors,vagus nerve
neuron type bolew unbranched 10,inferior vagus X ganglion,nucleus of solitary tract,vagus nerve
neuron type bolew unbranched 8,inferior vagus X ganglion,Aortic arch baroreceptors,vagus nerve
neuron type bolew unbranched 8,inferior vagus X ganglion,nucleus of solitary tract,vagus nerve
neuron type bolew unbranched 9,inferior vagus X ganglion,Laryngeal mechanoreceptors,vagus nerve
neuron type bolew unbranched 9,inferior vagus X ganglion,nucleus of solitary tract,vagus nerve


## Example 6: Neurons with All Connected Regions
This query returns a list of neuron populations along with their connected regions based on the their locational phenotypes. 

In [3]:
# Query: This query returns neuron populations with their connecting regions.
# The regions could be any of soma, axon, axon terminal, dendrite, or axon sensory terminal locations.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>


SELECT DISTINCT 
# ?Neuron_ID 
?Neuron_Label ?Connection_Type_Label ?Location_Label
{
  ## ***************************** INSTRUCTIONS **************************************** 
  ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
  ## REPLACE the region name on the right side of the equality operator (=) 
  ## by your desired region name within quotes (all in lower case).
  ## ***********************************************************************************
   
  ## For exact matching of the region name
  # FILTER (LCase(?Location_Label) = 'inferior vagus x ganglion')

  ## For partial matching of location name
  # FILTER REGEX (LCase(?Location_Label), 'vagus x')

  ## For partial matching of neuron label. Replace 'bromo' by other types.  
  #  FILTER REGEX (LCase(?Neuron_Label), 'bromo') 
    
    # We are only interested in connection forming neuron types i.e, neurons that form some connection 
    # between their origin and destination regions
    ?Neuron_ID ilxtr:hasSomaLocation ?A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B.

   
    ?Neuron_ID ilxtr:hasConnectedLocation ?Location_ID;
               rdfs:label ?Neuron_Label.
    
    ?Neuron_ID ?Connection_Type ?Location_ID.
    ?Location_ID rdfs:label ?Location_Label.
    ?Connection_Type rdfs:label ?Connection_Type_Label.

    
    FILTER (?Connection_Type != ilxtr:hasConnectedLocation)
}
ORDER BY ?Neuron_ID DESC(?Connection_Type) ?Location_Label
LIMIT 5000

Neuron_Label,Connection_Type_Label,Location_Label
neuron type aacar 10a,hasSomaLocation,Atrial intrinsic cardiac ganglion
neuron type aacar 10a,hasAxonTerminalLocation,Atrial intrinsic cardiac ganglion
neuron type aacar 10a,hasAxonTerminalLocation,ventricular intrinsic cardiac ganglion
neuron type aacar 10a,hasAxonLocation,cardiac interganglionic nerve
neuron type aacar 10v,hasSomaLocation,ventricular intrinsic cardiac ganglion
neuron type aacar 10v,hasAxonTerminalLocation,Atrial intrinsic cardiac ganglion
neuron type aacar 10v,hasAxonTerminalLocation,ventricular intrinsic cardiac ganglion
neuron type aacar 10v,hasAxonLocation,cardiac interganglionic nerve
neuron type aacar 11,hasSomaLocation,inferior vagus X ganglion
neuron type aacar 11,hasAxonLeadingToSensoryTerminal,vagus nerve


## Example 7: Neurons with Processes in IMG
This query can be used to answer "What anatomical structures might be affected by stimulating the inferior mesenteric ganglion?"
This example shows how to query neuron populations based on their connection to a particular region. We use inferior mesenteric ganglion (IMG) as an example of a given target region. 

In [10]:
# Query: What anatomical structures might be affected by perturbation of the IMG?
# This query returns the distinct set of anatomical structures connected to the IMG based 
# on a set of neuron populations in SCKAN with processes in IMG.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>

SELECT Distinct 
# ?Neuron_ID 
# ?Neuron_Label 
# ?Connection_Type 
?Location_Label
?Connected_Region
{
  ## ***************************** INSTRUCTIONS **************************************** 
  ## UNCOMMENT the lines by removing the hash symbol (#) before FILTER. 
  ## REPLACE the region name on the right side of the equality operator (=) 
  ## by your desired region name within quotes (all in lower case).
  ## ***********************************************************************************
  
    FILTER (LCase(?Location_Label) = 'inferior mesenteric ganglion')
   
    ?Neuron_ID ilxtr:hasConnectedLocation ?Location_ID.
    OPTIONAL {?Neuron_ID rdfs:label ?Neuron_Label.}
    
    ?Location_ID rdfs:label ?Location_Label.
    ?Neuron_ID ?Connection_Type ?Location_ID.
    
    ?Neuron_ID (ilxtr:hasSomaLocation | ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonLocation 
                |ilxtr:hasDendriteLocation | ilxtr:hasAxonSensoryLocation) ?Connected_Region_ID.
    
    ?Connected_Region_ID rdfs:label ?Connected_Region.
    
    # FILTER (?Connection_Type != ilxtr:hasConnectedLocation)
    FILTER (?Location_ID != ?Connected_Region_ID)

    # ?Location_ID ilxtr:isPartOf+/rdfs:label 'inferior mesenteric ganglion'.
  
}
ORDER BY ?Neuron_Label ?Connected_Region
LIMIT 1000

Location_Label,Connected_Region
inferior mesenteric ganglion,Arteriole in connective tissue of bladder dome
inferior mesenteric ganglion,Arteriole in connective tissue of bladder neck
inferior mesenteric ganglion,Dome of the Bladder
inferior mesenteric ganglion,L1 segment of lumbar spinal cord
inferior mesenteric ganglion,L2 segment of lumbar spinal cord
inferior mesenteric ganglion,first lumbar dorsal root ganglion
inferior mesenteric ganglion,hypogastric nerve
inferior mesenteric ganglion,neck of urinary bladder
inferior mesenteric ganglion,second lumbar dorsal root ganglion
inferior mesenteric ganglion,white matter of spinal cord


## Example 8: Neurons Targetting Major Organs
We often want to know list the neuron populations based on their targets in major organs. This query looks for the terminals of the populations that reside in any part of a major organ. In this example, we ask about the populations that terminate in any part of urinary bladder.   

In [16]:
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX partOf: <http://purl.obolibrary.org/obo/BFO_0000050>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>


SELECT DISTINCT
# ?Neuron_ID
?Neuron_Label 
?A_Label ?B_Label ?Target_Organ
{                
    
    ## For exact matching of the target organ name
    FILTER (LCase(?Target_Organ) = 'urinary bladder')

    ## For partial matching of the target organ name
    # FILTER REGEX (LCase(?Target_Organ), 'heart left')
    
    ?Neuron_ID ilxtr:hasSomaLocation ?A_IRI.
            ?A_IRI rdfs:label ?A_Label.
    ?Neuron_ID rdfs:label ?Neuron_Label.
   
    ?Neuron_ID (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?B_IRI.
                ?B_IRI rdfs:label ?B_Label.

    ?B_IRI ilxtr:isPartOf+/rdfs:label ?Target_Organ.
                    
    FILTER (?Target_Organ in ( 'heart', 'ovary', 'brain', 'urethra', 'esophagus', 'skin of body', 'lung', 'liver', 
                                'lower urinary tract', 'urinary tract', 'muscle organ','gallbladder', 'colon', 'kidney', 
                                'large intestine' ,'small intestine', 'stomach', 'spleen', 'urinary bladder', 
                                'penis', 'clitoris', 'pancreas'))       
}
ORDER BY ?Neuron_ID ?A_Label ?B_Label
LIMIT 9999

Neuron_Label,A_Label,B_Label,Target_Organ
neuron type kblad 1,pelvic ganglion,Dome of the Bladder,urinary bladder
neuron type kblad 1,pelvic ganglion,neck of urinary bladder,urinary bladder
neuron type kblad 10,first sacral dorsal root ganglion,Arteriole in connective tissue of bladder neck,urinary bladder
neuron type kblad 10,first sacral dorsal root ganglion,Dome of the Bladder,urinary bladder
neuron type kblad 10,first sacral dorsal root ganglion,neck of urinary bladder,urinary bladder
neuron type kblad 10,sixth lumbar dorsal root ganglion,Arteriole in connective tissue of bladder neck,urinary bladder
neuron type kblad 10,sixth lumbar dorsal root ganglion,Dome of the Bladder,urinary bladder
neuron type kblad 10,sixth lumbar dorsal root ganglion,neck of urinary bladder,urinary bladder
neuron type kblad 11,first lumbar dorsal root ganglion,Arteriole in connective tissue of bladder neck,urinary bladder
neuron type kblad 11,first lumbar dorsal root ganglion,Dome of the Bladder,urinary bladder


## Example 9: Neurons with Axonal Paths
This is a query where we want to produce an adjacency matrix to retrieve the axonal path of the neuron populations. Each axonal path forms a partial order of locations a neuron population forms between its origin(s) and destination(s). 
* <b>NOTE:</b> The result of this query can be used to visualize the axonal paths using different visualization tools. For example, you can use the result of this query to generate digraphs in dot format and visualize the axonal paths for the neuron populations in GraphViz. 


In [ ]:
# Query to generate the adjacency matrix representing the axonal paths of the neuron populations.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>


SELECT DISTINCT ?Neuron_IRI ?Neuron_Label ?V1 ?V1_Label ?V2 ?V2_Label
WHERE 
{

    # Note: This query currently works for the NLP curated neuron populations.
    # We will update the query for other populations such as the populations from ApINATOMY models. 
    ?Neuron_IRI rdfs:subClassOf ilxtr:NeuronSparcNlp.

    ?Neuron_IRI ilxtr:neuronPartialOrder ?PO .
        ?PO (rdf:rest|rdf:first)* ?r1 .
        ?r1 (rdf:rest|rdf:first)* ?r2 .
        ?r1 rdf:first ?V1 .
        ?r2 rdf:first ?V2 .
        ?V1 rdf:type owl:Class .
        ?V2 rdf:type owl:Class .

        FILTER (?mediator = ?r1)  # draw only from the same partial order
        ?mediator rdf:first ?V1 .  # car
        ?mediator rdf:rest+/rdf:first/rdf:first ?V2 .  # caadr

    ?V1 rdfs:label ?V1_Label. ?V2 rdfs:label ?V2_Label.
    ?Neuron_IRI rdfs:label ?Neuron_Label.

    FILTER (?V1 != ?V2).

   # FILTER (CONTAINS(STR(?Neuron_IRI), 'semves/2')).

}
ORDER BY ?Neuron_IRI ?V1 ?V2 
LIMIT 10000

Neuron_IRI,Neuron_Label,V1,V1_Label,V2,V2_Label
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0000996,vagina,http://purl.obolibrary.org/obo/UBERON_0018675,pelvic splanchnic nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0002261,dorsal root of spinal cord,http://purl.obolibrary.org/obo/UBERON_0006461,S2 segment of sacral spinal cord
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0002261,dorsal root of spinal cord,http://purl.obolibrary.org/obo/UBERON_0006462,S3 segment of sacral spinal cord
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0002261,dorsal root of spinal cord,http://purl.obolibrary.org/obo/UBERON_0006463,S4 segment of sacral spinal cord
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0002861,second sacral dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002261,dorsal root of spinal cord
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0002862,third sacral dorsal root ganglion,http://purl.obolibrary.org/obo/UBERON_0002261,dorsal root of spinal cord
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0007713,fourth sacral spinal ganglion,http://purl.obolibrary.org/obo/UBERON_0002261,dorsal root of spinal cord
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0018675,pelvic splanchnic nerve,http://purl.obolibrary.org/obo/UBERON_0002861,second sacral dorsal root ganglion
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0018675,pelvic splanchnic nerve,http://purl.obolibrary.org/obo/UBERON_0002862,third sacral dorsal root ganglion
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/femrep/100,neuron type femrep 100,http://purl.obolibrary.org/obo/UBERON_0018675,pelvic splanchnic nerve,http://purl.obolibrary.org/obo/UBERON_0007713,fourth sacral spinal ganglion


### Simpler, Alternative Query Using RDF Star
We have spacified RDF-star representation for the axonal path query above in Stardog, using ilxtr:hasNextNode predicate. However, RDF-star is not supported by Blazegraph. You will need to run the following query using Stardog endpoint, which is specified at the top of the query using magic. To save the results in json format simply add the following two magics befor the SPARQL code:
* `%format JSON`
* `%outfile filename.json`

In [10]:
%endpoint https://sd-c1e74c63.stardog.cloud:5820/NPO/query
%auth basic SPARC RCvp9tKzTdxg42py
%format any
%display table
%show 16

PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>

# Query to generate the adjacency table representing the axonal paths of the neuron populations in SCKAN.
SELECT DISTINCT ?Neuron_IRI ?Neuron_Label ?V1 ?V1_Label ?V2 ?V2_Label

# INSERT {?V1 ilxtr:hasNextNode {ilxtr:isConnectedBy ?Neuron_IRI} ?V2}
WHERE 
{
    ?Neuron_IRI rdfs:subClassOf* ilxtr:NeuronSparcNlp.
    
    # This is an RDF-star like syntax for stardog
    ?V1 ilxtr:hasNextNode {ilxtr:isConnectedBy ?Neuron_IRI} ?V2.

    ?V1 rdfs:label ?V1_Label. ?V2 rdfs:label ?V2_Label.
    ?Neuron_IRI rdfs:label ?Neuron_Label.

    FILTER (?V1 != ?V2).

  #  FILTER (CONTAINS(STR(?Neuron_IRI), 'semves/2')).

}
#ORDER BY ?Neuron_IRI ?V1 ?V2 
LIMIT 10000

Endpoint set to: https://sd-c1e74c63.stardog.cloud:5820/NPO/query
HTTP authentication: method=basic, user=SPARC, passwd set
Request format: ANY
Display: table
Result maximum size: 16

Neuron_IRI,Neuron_Label,V1,V1_Label,V2,V2_Label
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/114,neuron type liver 114,http://purl.obolibrary.org/obo/UBERON_0002017,portal vein,http://uri.interlex.org/base/ilx_0793826,anterior abdominal vagal trunk
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/113,neuron type liver 113,http://purl.obolibrary.org/obo/UBERON_0002017,portal vein,http://purl.obolibrary.org/obo/UBERON_0018679,thoracic splanchnic nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/126,neuron type liver 126,http://purl.obolibrary.org/obo/UBERON_0002017,portal vein,http://purl.obolibrary.org/obo/UBERON_0018679,thoracic splanchnic nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/115,neuron type liver 115,http://purl.obolibrary.org/obo/UBERON_0002017,portal vein,http://uri.interlex.org/base/ilx_0795011,periarterial neural plexus of the common hepatic artery
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/113,neuron type liver 113,http://purl.obolibrary.org/obo/UBERON_0002394,bile duct,http://purl.obolibrary.org/obo/UBERON_0018679,thoracic splanchnic nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/126,neuron type liver 126,http://purl.obolibrary.org/obo/UBERON_0002394,bile duct,http://purl.obolibrary.org/obo/UBERON_0018679,thoracic splanchnic nerve
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/115,neuron type liver 115,http://purl.obolibrary.org/obo/UBERON_0002394,bile duct,http://uri.interlex.org/base/ilx_0795011,periarterial neural plexus of the common hepatic artery
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/114,neuron type liver 114,http://purl.obolibrary.org/obo/UBERON_0002394,bile duct,http://uri.interlex.org/base/ilx_0793826,anterior abdominal vagal trunk
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/127,neuron type liver 127,http://purl.obolibrary.org/obo/UBERON_0002394,bile duct,http://uri.interlex.org/base/ilx_0793826,anterior abdominal vagal trunk
http://uri.interlex.org/tgbugs/uris/readable/sparc-nlp/liver/113,neuron type liver 113,http://purl.obolibrary.org/obo/UBERON_0018679,thoracic splanchnic nerve,http://purl.obolibrary.org/obo/UBERON_0002851,eighth thoracic dorsal root ganglion


## Example 10: Neurons with Forward Connections
We often need to know whether a neuron population is connected to other neuron populations through its synaptic terminal. SCKAN uses `ilxtr:hasForwardConnection` predicate to connect the two populations. Whenever we have such connection, that basically means that a preganglionic neuron population has a terminal location (i.e., axon terminal or axon sensory terminal) that is the origin of a postganglionic neuron population.

In [ ]:
# Query to list the neuron populations with their forward connections with other populations
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>


SELECT DISTINCT ?Neuron_1_IRI ?Neuron_1_Label 
                ?Neuron_2_IRI ?Neuron_2_Label
WHERE 
{
    # We are only interested in connection forming neuron types i.e, neurons that form some connection 
    # between their origin and destination regions
   ?Neuron_1_IRI ilxtr:hasSomaLocation ?Neuron_1_A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ?Nuron_1_B .
    
   ?Neuron_2_IRI ilxtr:hasSomaLocation ?Neuron_2_A;
               (ilxtr:hasAxonTerminalLocation | ilxtr:hasAxonSensoryLocation) ??Neuron_2_B .
    
   ## To restrict the results within the NLP curated neuron populations.
   #  ?Neuron_1_IRI rdfs:subClassOf ilxtr:NeuronSparcNlp .
   #  ?Neuron_2_IRI rdfs:subClassOf ilxtr:NeuronSparcNlp .

   
   ?Neuron_1_IRI ilxtr:hasForwardConnection ?Neuron_2_IRI .
   
   ?Neuron_1_IRI rdfs:label ?Neuron_1_Label .
   ?Neuron_2_IRI rdfs:label ?Neuron_2_Label . 

  ## Filter results based on a particular neuron population
  # FILTER(CONTAINS(STR(?Neuron_1_IRI), 'prostate')).  
  # FILTER(CONTAINS(STR(?Neuron_2_IRI), 'mmset4'))
   
} 
ORDER BY ?Neuron_1_IRI ?Neuron_2_IRI

Neuron_1_IRI,Neuron_1_Label,Neuron_2_IRI,Neuron_2_Label
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-7a,neuron type aacar 7a
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-8a,neuron type aacar 8a
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10a,neuron type aacar 10a
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-7v,neuron type aacar 7v
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-10v,neuron type aacar 10v,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-8v,neuron type aacar 8v
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-11,neuron type aacar 11,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-1,neuron type aacar 1
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-12,neuron type aacar 12,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2i,neuron type aacar 2i
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-12,neuron type aacar 12,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-2m,neuron type aacar 2m
http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-13,neuron type aacar 13,http://uri.interlex.org/tgbugs/uris/readable/neuron-type-aacar-14,neuron type aacar 14
